Test code to convery ncbi ids to uniprot

In [2]:
import pandas as pd
import numpy as np
import os
import collections


In [3]:
import mygene

mg = mygene.MyGeneInfo()

In [4]:
xli = ['DDX26B',
 'CCDC83',
 'MAST3',
 'FLOT1',
 'RPL11',
 'ZDHHC20',
 'LUC7L3',
 'SNORD49A',
 'CTSH',
 'ACOT8']
out = mg.querymany(xli, scopes='symbol', fields='entrezgene', species='human')
print out

Finished.
1 input query terms found no hit:
	[u'DDX26B']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
[{u'notfound': True, u'query': u'DDX26B'}, {u'query': u'CCDC83', u'entrezgene': 220047, u'_id': u'220047'}, {u'query': u'MAST3', u'entrezgene': 23031, u'_id': u'23031'}, {u'query': u'FLOT1', u'entrezgene': 10211, u'_id': u'10211'}, {u'query': u'RPL11', u'entrezgene': 6135, u'_id': u'6135'}, {u'query': u'ZDHHC20', u'entrezgene': 253832, u'_id': u'253832'}, {u'query': u'LUC7L3', u'entrezgene': 51747, u'_id': u'51747'}, {u'query': u'SNORD49A', u'entrezgene': 26800, u'_id': u'26800'}, {u'query': u'CTSH', u'entrezgene': 1512, u'_id': u'1512'}, {u'query': u'ACOT8', u'entrezgene': 10005, u'_id': u'10005'}]


In [5]:
xli = ["NM_001001177"]
out = mg.querymany(xli, scopes='refseq', fields='uniprot', species='mouse', returnall=True)
print out

Finished.
{'dup': [], 'missing': [], 'out': [{u'query': u'NM_001001177', u'_id': u'407788', u'uniprot': {u'TrEMBL': [u'A2CG24', u'A2CG25', u'A6X8K8', u'A6X8K9', u'Q3V0M8', u'Q80XD0', u'Q8BHP1']}}]}


In [6]:
wkdir = "/Users/toms/WT/NML"
data_dir = os.path.join(wkdir, "data")

In [157]:
scaled_df = pd.read_table(os.path.join(data_dir, "GSE73467_Scaled_data.csv"), sep=",")
scaled_df.to_csv(os.path.join(data_dir, "GSE73467_Scaled_data.tsv"), sep="\t", index=False)
scaled_df = scaled_df.iloc[:,1:6]
print scaled_df.head(2)

            Gene  Scaled bc05 (E1)  Scaled bc06 (E2)  Scaled bc07 (E3)  \
0  0610005C13Rik          0.074308          0.088382               0.0   
1  0610005C13Rik          0.000003          0.000003               0.0   

   Scaled bc08 (E4)  
0               0.0  
1               0.0  


In [8]:
out = mg.querymany(scaled_df['Transcript'].tolist(), scopes='refseq', fields='uniprot', species='mouse', returnall=True)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-27796...done.
Finished.
524 input query terms found no hit:
	[u'NM_001177543.1', u'NM_001177543.2', u'NM_001177543.3', u'NM_001177543.4', u'NM_001177543.5', u'NR


We only want to match the uniprot ids found in the hyperLOPIT data. So let's start by extracting these ids.

In [424]:
lopit_df = pd.read_table(os.path.join(data_dir, "ncomms.tsv.txt"), sep="\t")

lopit_df['uniprot_id'] = [x.split("-")[0] for x in lopit_df['UniProt Accession for Protein Group']]
print len(lopit_df['uniprot_id'])
print lopit_df.shape

lopit_df.drop_duplicates(subset=['uniprot_id'], inplace=True)
print lopit_df.shape

5032
(5032, 46)
(5011, 46)


In [159]:
query2uniprot = {}

for query in out['out']:
    if "uniprot" in query:
        if 'Swiss-Prot' in query['uniprot']:
            n = 0
            matches = []
            for test_id in query['uniprot']['Swiss-Prot']:
                if test_id in lopit_uniprots:
                    n+=1
                    matches.append(test_id)
            if n>1:
                print query
                print matches
                break
            #if type(query['uniprot']['Swiss-Prot']) is list:
                #print query, query['query'], query['uniprot']['Swiss-Prot']
                #break
            query2uniprot[query['query']] = query['uniprot']['Swiss-Prot']

#print query2uniprot

{u'query': u'NM_145990', u'_id': u'214444', u'uniprot': {u'Swiss-Prot': [u'P68254', u'Q8K389'], u'TrEMBL': [u'A3KML3', u'F6R2K5', u'F6VW30', u'F6YY69', u'H7BX43', u'Q0VGR5', u'Q5SP75']}}
[u'P68254', u'Q8K389']


OK, so the conversion doesn't work well. Above we have a ncbi transcript id "NM_145990" which apparently maps to 2 Swiss-Prot ids [u'P68254', u'Q8K389'], both of which are in the LOPIT data. Below, we search through the entire set of ncbi transcript ids in the file and find that NM_011739 also matches P68254. Indeed according to uniprot converter, NM_145990 is Q8K389 and NM_011739 is P68254. So it looks like we're better off using the uniprot converter.

In [160]:
for query in out['out']:
    if "uniprot" in query:
        if 'Swiss-Prot' in query['uniprot']:
            n = 0
            matches = []
            if type(query['uniprot']['Swiss-Prot']) == list:
                for test_id in query['uniprot']['Swiss-Prot']:
                    if test_id == "P68254" or test_id == "Q8K389":
                        print query
            else:
                if query['uniprot']['Swiss-Prot'] == "P68254" or query['uniprot']['Swiss-Prot'] == "Q8K389":
                    print query

{u'query': u'NM_145990', u'_id': u'214444', u'uniprot': {u'Swiss-Prot': [u'P68254', u'Q8K389'], u'TrEMBL': [u'A3KML3', u'F6R2K5', u'F6VW30', u'F6YY69', u'H7BX43', u'Q0VGR5', u'Q5SP75']}}
{u'query': u'NM_145990', u'_id': u'214444', u'uniprot': {u'Swiss-Prot': [u'P68254', u'Q8K389'], u'TrEMBL': [u'A3KML3', u'F6R2K5', u'F6VW30', u'F6YY69', u'H7BX43', u'Q0VGR5', u'Q5SP75']}}
{u'query': u'NM_011739', u'_id': u'22630', u'uniprot': {u'Swiss-Prot': u'P68254', u'TrEMBL': [u'A3KML3', u'F6VW30', u'F6YY69']}}


Below, we have the dataframe from the coversion:

In [161]:
uniprot_df = pd.read_table(os.path.join(data_dir, "uniprot_convert.tsv"), sep="\t", index_col=0)
uniprot_df.index.name = "ncbi_id"
uniprot_df = uniprot_df[uniprot_df['Status']=="reviewed"]
print uniprot_df.head(2)
print uniprot_df.shape

t_id_counts = collections.Counter(uniprot_df.index)

print uniprot_df.ix['NM_145412']



             isomap:M20161122AAFB7E4D2F1D05654627429E83DA5CCE998041Y   Entry  \
ncbi_id                                                                        
NM_001001152                                                NaN       Q6P5C7   
NM_001001160                                                NaN       Q6NS60   

               Entry name    Status            Protein names  \
ncbi_id                                                        
NM_001001152  ZN728_MOUSE  reviewed  Zinc finger protein 728   
NM_001001160  FBX41_MOUSE  reviewed    F-box only protein 41   

                              Gene names              Organism  Length  
ncbi_id                                                                 
NM_001001152               Znf728 Zfp458  Mus musculus (Mouse)     752  
NM_001001160  Fbxo41 D6Ertd538e Kiaa1940  Mus musculus (Mouse)     873  
(15093, 8)
          isomap:M20161122AAFB7E4D2F1D05654627429E83DA5CCE998041Y   Entry  \
ncbi_id                                   

Ok, so there are 8 transcript ids (or sets of transcript ids) which map to multiple proteins. Let's just ignore these to make life simplier

In [162]:
n = 0
for t_id in uniprot_df.index:
    if t_id_counts[t_id]>1:
        n+=1
        print t_id, t_id_counts[t_id]
        print uniprot_df.ix[t_id]
        
print n
    

NM_001164787,NM_011468 2
                       isomap:M20161122AAFB7E4D2F1D05654627429E83DA5CCE998041Y  \
ncbi_id                                                                          
NM_001164787,NM_011468                                                  ,        
NM_001164787,NM_011468                                                  ,        

                         Entry   Entry name    Status  \
ncbi_id                                                 
NM_001164787,NM_011468  Q4KL71  SP2A3_MOUSE  reviewed   
NM_001164787,NM_011468  Q9CQK8  SPR2A_MOUSE  reviewed   

                                         Protein names Gene names  \
ncbi_id                                                             
NM_001164787,NM_011468  Small proline-rich protein 2A3    Sprr2a3   
NM_001164787,NM_011468   Small proline-rich protein 2A     Sprr2a   

                                    Organism  Length  
ncbi_id                                               
NM_001164787,NM_011468  Mus mu

In [447]:
t_id2uniprot = {}
t_id2length = {}

for t_id in uniprot_df.index:
    if t_id_counts[t_id]>1:
        continue
    for t_split_id in t_id.split(","):
        t_id2uniprot[t_split_id] = uniprot_df.ix[t_id]['Entry']
        t_id2length[t_split_id] = uniprot_df.ix[t_id]['Length'] 
        
    

We now have a dictionary to convert ncbi id to uniprot id. we will use this method in the NML_multiple_isoforms notebook